In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:

import altair as alt

from tools import get_all_files, get_GHG_file_df
from tools.Plot_GHG import get_GHG_plots

In [14]:
# Get the root directory of the data
data_root = '../2023_11_03__17_49_42_hard_mincost_RF30_P1e4_Won_Gon'

# Get all output files
files = get_all_files(data_root)

In [15]:
# Select files based on criteria
files_selc = files.query('year <= 2050')
print(files_selc['catetory'].unique(),'\n')

files_selc

['GHG' 'dvar' 'ammap' 'lmmap' 'lumap_separate' 'lumap' 'water'
 'cross_table' 'quantity'] 



,year,catetory,base_name,base_ext,path
0,2010,GHG,GHG_emissions,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
1,2010,GHG,GHG_emissions_separate_agricultural_landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
2,2010,GHG,GHG_emissions_separate_agricultural_management,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
3,2010,GHG,GHG_emissions_separate_no_ag_reduction,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
4,2010,GHG,GHG_emissions_separate_transition_penalty,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
...,...,...,...,...,...
2482,2050,cross_table,switches-irrstat,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
2483,2050,cross_table,switches-lmmap,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
2484,2050,cross_table,switches-lumap,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
2485,2050,cross_table,switches-precision-agriculture-amstat,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...


## GHG emissions

In [16]:
# Get the GHG files
GHG_files = get_GHG_file_df(files_selc)
GHG_files = GHG_files.reset_index(drop=True).sort_values(['year','GHG_sum_t'])
GHG_files['GHG_sum_Mt'] = GHG_files['GHG_sum_t'] / 1e6
GHG_files.head(3)

,year,catetory,base_name,base_ext,path,GHG_sum_t,GHG_sum_Mt
1,2010,GHG,Agricultural Management,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,0.0,0.0
2,2010,GHG,Non-Agricultural Landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,0.0,0.0
3,2010,GHG,Transition Penalty,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,0.0,0.0


In [17]:
# Calculate the net emissions
Net_emission = GHG_files.groupby('year')['GHG_sum_Mt'].sum(numeric_only = True).reset_index()
Net_emission = Net_emission.rename(columns={'GHG_sum_Mt':'Net_emission'})
Net_emission.head(3)

,year,Net_emission
0,2010,36.820685
1,2011,-70.476159
2,2012,-70.124035


### 0) Total emissions -- Column chart

In [18]:
# Create a base chart with the necessary transformations and encodings
base_chart = alt.Chart(GHG_files).transform_calculate(
    GHG_sum_Mt = "datum.GHG_sum_t/1000000"
).encode(
    x=alt.X('year:O',axis=alt.Axis(title="Year", labelAngle=-90)),  # Treat year as an ordinal data type
    tooltip=[alt.Tooltip('base_name', title='GHG Category'),
             alt.Tooltip('GHG_sum_Mt:Q', title='Emissions (Mt CO2e)')]
).properties(
    width=600,
    height=400
)



# Create a column chart with the base chart
column_chart = base_chart.mark_bar().encode(
    color=alt.Color('base_name:N',legend=alt.Legend(
                                            title="GHG Category",
                                            orient='none',
                                            legendX=130, legendY=-40,
                                            direction='horizontal',
                                            titleAnchor='middle')),  
    y=alt.Y('GHG_sum_Mt:Q',title='Emissions (Mt CO2e)'),  # Treat GHA_accumulative as a quantitative field
)

# Create a column chart with the base chart
line_chart = alt.Chart(Net_emission).mark_line(color='black').encode(
    tooltip=[alt.Tooltip('Net_emission:Q',title='Net Quantity (Mt CO2e)')],
    x=alt.X('year:O'),
    y=alt.Y('Net_emission:Q',title='Net Quantity (Mt CO2e)'))


# Combine the layers into a final chart
final_chart = alt.layer(
    column_chart,
    line_chart,
).properties(
    width=800,
    height=450
)


final_chart


alt.LayerChart(...)

### 1) Agricultural Land use Emissions

In [85]:
# Initialize the class
ag_lucc_GHG = get_GHG_plots(GHG_files,'Agricultural Landuse','GHG emissions (Mt CO2e)')

  0%|          | 0/41 [00:00<?, ?it/s]

#### 1-1) Emissions by Crops/Lvstk -- Stack column

In [21]:
# get the df and plot
GHG_crop_lvstk_df,GHG_crop_lvstk_plot = ag_lucc_GHG.plot_GHG_crop_lvstk()
GHG_crop_lvstk_plot

alt.LayerChart(...)

#### 1-2) Emissions by dry/irr -- Stack columns

In [22]:
# get the df and plot
GHG_dry_irr_df,GHG_dry_irr_plot = ag_lucc_GHG.plot_GHG_dry_irr()
GHG_dry_irr_plot

alt.LayerChart(...)

In [87]:
ag_lucc_GHG.GHG_df_long

,Year,Land use category,Land use,Irrigation,Sources,Quantity (Mt CO2e),GHG Category
0,2010,Crop,Citrus,dry,TCO2E_CHEM_APPL,0.0,CO2
1,2010,Crop,Cotton,dry,TCO2E_CHEM_APPL,0.019594,CO2
2,2010,Crop,Grapes,dry,TCO2E_CHEM_APPL,0.0,CO2
3,2010,Crop,Hay,dry,TCO2E_CHEM_APPL,0.018938,CO2
4,2010,Crop,Nuts,dry,TCO2E_CHEM_APPL,0.018463,CO2
...,...,...,...,...,...,...,...
22711,2050,Crop,Winter legumes,irr,TCO2E_ENTERIC,0.0,CH4
22712,2050,Crop,Winter oilseeds,irr,TCO2E_ENTERIC,0.0,CH4
22713,2050,Livestock,Beef,irr,TCO2E_ENTERIC,0.333733,CH4
22714,2050,Livestock,Dairy,irr,TCO2E_ENTERIC,0.121018,CH4


#### 1-3) Emissions by GHG category

In [112]:
# get the df and plot
GHG_rice_df,GHG_rice_plot = ag_lucc_GHG.plot_GHG_category()
GHG_rice_plot

alt.LayerChart(...)

#### 1-4) Emissions by lu-lm cmobined

In [23]:
# get the df and plot
GHG_lu_lm_df,GHG_lu_lm_plot = ag_lucc_GHG.plot_GHG_lu_lm(2050)
GHG_lu_lm_plot

alt.Chart(...)

#### 1-5) Emissions by lu-sources combined

In [24]:
# get the df and plot
GHG_lu_source_df,GHG_lu_source_plot = ag_lucc_GHG.plot_GHG_lu_source(2050)
GHG_lu_source_plot

alt.Chart(...)

In [55]:
from PARAMETERS import GHG_CATEGORY
GHG_CATEGORY

{'TCO2E_DUNG_URINE': {'CH4': 0.5, 'CO2': 0.5},
 'TCO2E_ENTERIC': {'CH4': 1},
 'TCO2E_SOIL': {'N2O': 1}}

In [42]:
GHG_lu_source_df

,Year,Land use,Irrigation,Sources,Quantity (Mt CO2e)
20944,2050,Beef,dry,TCO2E_CHEM_APPL,0.014446
20945,2050,Beef,dry,TCO2E_CROP_MGT,0.001798
20946,2050,Beef,dry,TCO2E_CULTIV,0.015247
20947,2050,Beef,dry,TCO2E_DUNG_URINE,0.261877
20948,2050,Beef,dry,TCO2E_ELEC,0.000000
...,...,...,...,...,...
21449,2050,Winter oilseeds,irr,TCO2E_MANURE_MGT,0.000000
21450,2050,Winter oilseeds,irr,TCO2E_PEST_PROD,0.000000
21451,2050,Winter oilseeds,irr,TCO2E_SEED,0.000000
21452,2050,Winter oilseeds,irr,TCO2E_SOIL,0.000000


In [71]:
def get_GHG_category(GHG_lu_source_df):

    # 1) get CO2 GHG
    GHG_CO2 = GHG_lu_source_df.query('~Sources.isin(@GHG_CATEGORY.keys())').copy()
    GHG_CO2.loc['GHG Category'] = 'CO2'

    # 2) get non-CO2 GHG
    GHG_nonCO2 = GHG_lu_source_df.query('Sources.isin(@GHG_CATEGORY.keys())').copy()
    GHG_nonCO2['GHG Category'] = GHG_nonCO2['Sources'].apply(lambda x: GHG_CATEGORY[x].keys())
    GHG_nonCO2['Multiplier'] = GHG_nonCO2['Sources'].apply(lambda x: GHG_CATEGORY[x].values())
    GHG_nonCO2 = GHG_nonCO2.explode(['GHG Category','Multiplier']).reset_index(drop=True)
    GHG_nonCO2['Quantity (Mt CO2e)'] = GHG_nonCO2['Quantity (Mt CO2e)'] * GHG_nonCO2['Multiplier']

    return pd.concat([GHG_CO2,GHG_nonCO2],axis=0).reset_index(drop=True)

In [56]:
GHG_CATEGORY

{'TCO2E_DUNG_URINE': {'CH4': 0.5, 'CO2': 0.5},
 'TCO2E_ENTERIC': {'CH4': 1},
 'TCO2E_SOIL': {'N2O': 1}}

### 2) Non-agricultural Land use sequestrations

In [25]:
# Initialize the class
non_ag_lucc_GHG = get_GHG_plots(GHG_files,'Non-Agricultural Landuse','GHG sequestration (Mt CO2e)')

  0%|          | 0/41 [00:00<?, ?it/s]

#### 2-1) Sequestrations in total -- line chart

In [36]:
# get the df and plot
GHG_non_ag_crop_lvstk_df,GHG_non_ag_crop_lvstk_plot = non_ag_lucc_GHG.plot_GHG_crop_lvstk()
GHG_non_ag_crop_lvstk_plot

alt.LayerChart(...)

#### 2-2) Sequestrations by dry/irr -- stack columns

In [27]:
# get the df and plot
GHG_non_ag_dry_irr_df,GHG_non_ag_dry_irr_plot = non_ag_lucc_GHG.plot_GHG_dry_irr()
GHG_non_ag_dry_irr_plot

alt.LayerChart(...)